<a href="https://colab.research.google.com/github/Lorenzo-Gardini/DeepLearnig/blob/main/DeepLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Authors
The authors of this code, this notebook and its documentation are [Lorenzo Gardini](mailto: lorenzo.gardini7@studio.unibo.it) and [Vlad Mattiussi](mailto: vlad.mattiussi@studio.unibo.it).

prendi spunto da https://colab.research.google.com/drive/1f9BxFxPxhPKUba8Ifcq4juv042ZA484-

In [1]:
%autosave 60

Autosaving every 60 seconds


In [17]:
%%capture
!pip install stable-baselines3[extra]
!pip install gym[box2d]==0.26.0
!pip install gym
!pip install gym pyvirtualdisplay
!pip install imageio-ffmpeg

# **Useful modules import**
First of all, it is necessary to import useful modules used during the tutorial.

# Utility functions

In [8]:
import gym
from gym.wrappers import RecordVideo

VIDEO_FOLDER = "./video"

def make_env():
  return gym.make("ALE/Freeway-v5")

def make_video_env():
  env = gym.make("ALE/Freeway-v5", render_mode="rgb_array")
  return RecordVideo(env, VIDEO_FOLDER)

def play_env(policy):
  env = make_env()
  observation, _ = env.reset()
  cumulated_rewards = 0

  for i in range(5000):
    observation, reward, terminated, truncated, info = env.step(policy(observation))
    cumulated_rewards += reward

  env.close()

In [ ]:
play_env(lambda _: make_env().action_space.sample())

[MoviePy] >>>> Building video /content/video/rl-video-episode-0.mp4
[MoviePy] Writing video /content/video/rl-video-episode-0.mp4


100%|██████████| 1001/1001 [00:12<00:00, 81.45it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: /content/video/rl-video-episode-0.mp4 

[MoviePy] >>>> Building video /content/video/rl-video-episode-0.mp4
[MoviePy] Writing video /content/video/rl-video-episode-0.mp4


100%|██████████| 1001/1001 [00:11<00:00, 84.18it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: /content/video/rl-video-episode-0.mp4 

[MoviePy] >>>> Building video /content/video/rl-video-episode-0.mp4
[MoviePy] Writing video /content/video/rl-video-episode-0.mp4


100%|██████████| 1001/1001 [00:07<00:00, 129.95it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: /content/video/rl-video-episode-0.mp4 



In [ ]:
from IPython.display import HTML
from base64 import b64encode

mp4 = open("video/rl-video-episode-0.mp4",'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

HTML(""" <video controls autoplay><source src="%s" type="video/mp4"></video>""" % data_url)

## PPO

In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
"""
env = make_vec_env("CarRacing-v2", n_envs=4)

model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=25000)
model.save("ppo_cartpole")
"""

Using cpu device
Wrapping the env in a VecTransposeImage.


ValueError: ignored

## DQN
(action space incompatibile: dqn richiede solo discreto mentre l'env utilizza box.)

In [ ]:
#!pip install stable-baselines3[extra] pyvirtualdisplay gym[box2d] pyglet
#from pyvirtualdisplay import Display
#display = Display()
#display.start()

In [23]:
env = gym.make("ALE/Freeway-v5")

AttributeError: ignored

In [ ]:
envs = gym.envs.registry.all()
print(envs)
print('Total envs available:', len(envs))

In [ ]:
from stable_baselines3 import DQN

model = DQN("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10000, log_interval=4)
model.save("CarRacing-v2")

del model # remove to demonstrate saving and loading

model = DQN.load("CarRacing-v2")

obs = env.reset()
while True:
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)
    env.render()
    if done:
      obs = env.reset()

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
